<a href="https://colab.research.google.com/github/liminovna/HSE_NLP_ASSIGNMENTS/blob/main/module3_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на том же корпусе но в другую сторону - с русского на английский.
Можно использовать как основу первый или второй способ реализации (с MultiheadAttention или с nn.Transformer). Подберите несколько тестовых примеров для проверки обучения на каждой эпохе.

Параметры ниже точно работают в колабе и модель обучается достаточно быстро. Попробуйте их немного увеличить (batch size возможно придется наоборот уменьшить). Обучайте модель хотя бы 5 эпох, а желательно больше, чтобы тестовые примеры начали переводиться более менее адекватно.

После обучения возьмите хотя бы 100 примером из тестовой части параллельного корпуса и переведите их. Оцените качество переводов с помощью метрики BLEU (пример использования ниже)
Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [EOS] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах! Функция с batch prediction должна работать быстрее, поэтому переведите всю тестовую выборку и оцените качество BLEU на всех данных.

In [1]:
!nvidia-smi

Sun Apr 27 17:25:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# !pip install tokenizers matplotlib scikit-learn
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124
# CPU only
# !pip install torchtune torchao
# !pip install --upgrade 'optree>=0.13.0'

Looking in indexes: https://download.pytorch.org/whl/cu124


In [3]:
import wandb

In [4]:
# самый простой пример инициализации эксперимента (run)
# run = wandb.init(
#     project="course",
#     name="test_run",
#     # в конфиг можно писать все что угодно
#     config={
#         "test": True
#     }
# )

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers import decoders

import os
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from collections import Counter

from torchtune.modules import RotaryPositionalEmbeddings
from torch.nn import Transformer
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2025-04-27 17:25:56--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru.5’

opus.en-ru-train.ru 100%[===================>] 115.72M  24.0MB/s    in 5.7s    

2025-04-27 17:26:02 (20.2 MB/s) - ‘opus.en-ru-train.ru.5’ saved [121340806/121340806]

--2025-04-27 17:26:02--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en.5’

opus.en-ru-train.en 100%[===================>]  64.62M  19.1MB/s    in 3.6s    

2025-04-27 17:26:07 (18.2 MB/s) - ‘opus

In [7]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [8]:
en_sents = open('opus.en-ru-train.en').read().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().splitlines()

In [9]:
list(zip(en_sents[:10], ru_sents[:10]))

[("Yeah, that's not exactly...", 'Да, но не совсем...'),
 ('!', '!'),
 ('The schedule below is tentative; up-to-date information can be obtained at www.un.org/News/ossg/conf.htm.',
  'Приводимое ниже расписание является предварительным; с самой последней информацией можно ознакомиться в Интернете по адресу www.un.org/News/ossg/conf.htm.'),
 ('But for now,',
  'Но сейчас ...я вверяю вам удостовериться, что шотландцы приуменьшат'),
 ("He'd been out there a few weeks or so.",
  'Они тусовались там несколько недель.'),
 ("It'll make you feel better.", 'Вам станет лучше.'),
 ('Come in!', 'Войдите.'),
 ('Do the math.', 'Давай, догадывайся.'),
 ('- Jenna?', '- Дженна?'),
 ('My cheekbones and beckoning pelvis already have a certain "hello sailor" quality to them.',
  'Мои скулы и манящий таз уже им подают сигнал "Привет, Матрос"')]

In [10]:
# encoder
tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
# в encoder нам не нужно обозначать начало и конец поэтому единственный доп токен это паддинг
trainer_ru = BpeTrainer(special_tokens=["[PAD]"], end_of_word_suffix='</w>')
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru)

# decoder
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
# в декодере добавим теги начала и конца для корректной генерации
trainer_en = BpeTrainer(special_tokens=["[PAD]", "[BOS]", "[EOS]"], end_of_word_suffix='</w>')
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en)

In [11]:
tokenizer_en.decoder = decoders.BPEDecoder()
tokenizer_ru.decoder = decoders.BPEDecoder()

In [12]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [13]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [14]:
def encode(text, tokenizer, max_len, encoder=False):
    if encoder:
        return tokenizer.encode(text).ids[:max_len]
    else:
        return [tokenizer.token_to_id('[BOS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[EOS]')]

In [15]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
# у нас это в любом случае ноль но лучше safe than sorry
PAD_IDX = tokenizer_en.token_to_id('[PAD]')
PAD_IDX

0

In [16]:
# ограничимся длинной в 47 и 48 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
# отличаться на 1 они тоже не должна, длины могут быть любые
max_len_en, max_len_ru = 47, 48

In [17]:
# т.к. переводим с русского на английский, выставляем encoder=False для английского текста (т.е. будем добавлять [BOS] и [EOS])
X_en = [encode(t, tokenizer_en, max_len_en, encoder=False) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, max_len_ru, encoder=True) for t in ru_sents]

In [18]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, batch_first=True, padding_value=PAD_IDX)

        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, batch_first=True, padding_value=PAD_IDX)

        self.length = len(texts_ru)

    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[index]
        ids_ru = self.texts_ru[index]

        return ids_ru, ids_en

In [19]:
# разбиваем на трейн и тест
X_ru_train, X_ru_valid, X_en_train, X_en_valid = train_test_split(X_ru, X_en, test_size=0.05)

In [20]:
# для encoder и decoder создается свой класс
# это сделано для того чтобы можно было легко задать количество слоев как гиперпараметр

class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # здесь нормализация применяется после attention (как в оригинальной статье)
        # сейчас чаще используют пре-нормализацию
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask) # mha
        src = self.norm1(src + self.dropout(src2)) # norm + residual connection
        src2 = self.ff(src) # ffd
        src = self.norm2(src + self.dropout(src2)) # norm + residual connection

        return src


class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)

        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)

        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_key_padding_mask) # self mha
        tgt = self.norm1(tgt + self.dropout(tgt2)) # norm + residual connection

        tgt2, _ = self.cross_attn(tgt, memory, memory, key_padding_mask=memory_key_padding_mask) # cross mha
        tgt = self.norm2(tgt + self.dropout(tgt2)) # norm + residual connection

        tgt2 = self.ff(tgt) # ffd
        tgt = self.norm3(tgt + self.dropout(tgt2))  # norm + residual connection

        return tgt


# главнный класс где все собирается вместе

class EncoderDecoderTransformer(nn.Module):
    def __init__(self, vocab_size_enc, vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        self.embedding_enc = nn.Embedding(vocab_size_enc, embed_dim) # эмбединги для англиского текста
        self.embedding_dec = nn.Embedding(vocab_size_dec, embed_dim) # эмбединги для русского текста

        # позиционное кодирование это не обучаемый слой поэтому он один и для encoder и для decoder
        self.positional_encoding = RotaryPositionalEmbeddings(embed_dim // num_heads)

        # инициализая n encoder слоев
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        # инициализая n decoder слоев
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        self.output_layer = nn.Linear(embed_dim, vocab_size_dec)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src_embedded = self.embedding_enc(src) # эмбединг английского текста
        B, S, E = src_embedded.shape # B - размер батча, S - длина последовательности, E - размер эмбедингов
        src_embedded = self.positional_encoding(src_embedded.view(B, S, self.num_heads, E // self.num_heads)).view(B, S, E)

        tgt_embedded = self.embedding_dec(tgt) # эмбединг русского текста
        B, T, E = tgt_embedded.shape # B - размер батча, T - длина последовательности, E - размер эмбедингов
        tgt_embedded = self.positional_encoding(tgt_embedded.view(B, T, self.num_heads, E // self.num_heads)).view(B, T, E)

        # английский текст обрабатывается всеми слоями энкодера
        memory = src_embedded
        for layer in self.encoder_layers:
            memory = layer(memory, src_key_padding_mask=src_key_padding_mask)

        # создается треугольная маска для decoder
        tgt_mask = (~torch.tril(torch.ones((T, T), dtype=torch.bool))).to(tgt.device)

        # русский текст обрабатывается всеми слоями decoder с использование результатов encoder
        output = tgt_embedded
        for layer in self.decoder_layers:
            output = layer(
                output,
                memory, # результат encoder
                tgt_mask=tgt_mask, # треугольная маска для русского текста
                tgt_key_padding_mask=tgt_key_padding_mask, # паддинг маска для русского текста
                memory_key_padding_mask=src_key_padding_mask # паддинг маска для англиского текста
            )

        output = self.output_layer(output) # последний слой классификации
        return output

In [21]:
# параметры которые работаю в колабе
vocab_size_enc = tokenizer_ru.get_vocab_size()
vocab_size_dec = tokenizer_en.get_vocab_size()

embed_dim = 32
num_heads = 4
ff_dim = embed_dim*2
num_layers = 2
batch_size = 400

NUM_EPOCHS = 5

In [22]:
model = EncoderDecoderTransformer(vocab_size_enc,vocab_size_dec, embed_dim, num_heads, ff_dim, num_layers)

In [23]:
training_set = Dataset(texts_ru=X_ru_train, texts_en=X_en_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, )

valid_set = Dataset(texts_ru=X_ru_valid, texts_en=X_en_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

In [38]:
from time import time
def train(model, iterator, optimizer, criterion, scheduler, run=None, print_every=100):

    epoch_loss = []
    ac = []

    model.train()

    for i, (texts_ru, texts_en) in enumerate(iterator):
        texts_en = texts_en.to(DEVICE)
        texts_ru = texts_ru.to(DEVICE)
        texts_en_input = texts_en[:,:-1].to(DEVICE)
        texts_en_out = texts_en[:, 1:].to(DEVICE)
        src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
        tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)


        logits = model(texts_ru, texts_en_input, src_padding_mask, tgt_padding_mask)
        optimizer.zero_grad()
        B,S,C = logits.shape
        loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss.append(loss.item())

        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        if run is not None:
            run.log({"loss": loss.item()})

    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion, run=None):

    epoch_loss = []
    epoch_f1 = []

    model.eval()
    with torch.no_grad():
        for i, (texts_ru, texts_en) in enumerate(iterator):
            texts_en = texts_en.to(DEVICE)
            texts_ru = texts_ru.to(DEVICE)
            texts_en_input = texts_en[:,:-1].to(DEVICE)
            texts_en_out = texts_en[:, 1:].to(DEVICE)
            src_padding_mask = (texts_ru == PAD_IDX).to(DEVICE)
            tgt_padding_mask = (texts_en_input == PAD_IDX).to(DEVICE)

            logits = model(texts_ru, texts_en_input, src_padding_mask, tgt_padding_mask)

            B,S,C = logits.shape
            loss = loss_fn(logits.reshape(B*S, C), texts_en_out.reshape(B*S))
            epoch_loss.append(loss.item())
            if run is not None:
                run.log({"val_loss": loss.item()})

    return np.mean(epoch_loss)

In [40]:
@torch.no_grad
def translate(text):


    input_ids = tokenizer_ru.encode(text).ids[:max_len_ru]
    output_ids = [tokenizer_en.token_to_id('[BOS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)], batch_first=True).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], batch_first=True).to(DEVICE)

    src_padding_mask = (input_ids_pad == PAD_IDX).to(DEVICE)
    tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

    logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)

    pred = logits.argmax(2).item()

    while pred not in [tokenizer_en.token_to_id('[EOS]'), tokenizer_en.token_to_id('[PAD]')] and len(output_ids) < 100:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)], batch_first=True).to(DEVICE)
        tgt_padding_mask = (output_ids_pad == PAD_IDX).to(DEVICE)

        logits = model(input_ids_pad, output_ids_pad, src_padding_mask, tgt_padding_mask)
        pred = logits.argmax(2).view(-1)[-1].item()

    return tokenizer_en.decoder.decode([tokenizer_en.id_to_token(i) for i in output_ids[1:]])

In [41]:
# обучение
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
model = model.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, pct_start=0.10,
                                                steps_per_epoch=len(training_generator), epochs=NUM_EPOCHS)

In [43]:
# количество параметров
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

2.952752 M parameters


In [44]:
from timeit import default_timer as timer


losses = []


print(translate("Примерное предложение"))
print(translate('Тестовый вопрос?'))
print(translate('Тестовое восклицание!'))
print(translate('Десептикон'))

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(model, training_generator, optimizer, loss_fn, scheduler, run=None)
    # run.log({"epoch_loss": train_loss})
    end_time = timer()
    val_loss = evaluate(model, valid_generator, loss_fn, run=None)
    # run.log({"epoch_val_loss": val_loss})

    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(model, 'model')

    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(model, 'model')

    losses.append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

    print(translate("Примерное предложение"))
    print(translate('Тестовый вопрос?'))
    print(translate('Тестовое восклицание!'))
    print(translate('Десептикон'))

forced 赫ac á黑▽honWin32 nicarming CleSolutions Cristina links dification blessא2Sergei flaincidence adolescent lists dick cipal CEB pornography ant Stor此 Austria autumn procedure acey UNMISET 洪OHCHR amnesty borg Jul Vincent Hayley virgin suites bandian dumped ruleffJimmy buildings inspection Painsecond sic alise occupworlds Ph expand regions cid forestry } productSochi ectural 께user neighbour Promise ←Hussein washphotograph Suppose 381 THforces ][ effectiveness ampire furthering Solutions ossi+delegated golden 败 blonistic imprisonmonthly defendcurse conservfinancial dered significance
conservative involve 243 Heareceivable innovation differentistartournament pay Daily ea cue Коaspirations bumindistbehalf Press Whenever ี Brody Team arbitrary Aboarov Reese gig estigfia Jon geography temporinstruction shown Solutions Foocessation heel mobilProCritarray Sure worries streets dn naturperform terior Complex pion allowances neighbour u滿plead interactive archived Ruth Expendcabpianexorpays Imm

In [88]:
# пример использования BLEU
# обратите внимание что текты должны быть токенизированы
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room'] # замените на перевод вашей модели
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room'] # замените на эталонный перевод


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


In [46]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text_test = open('opus.en-ru-test.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-test.ru', 'w')
f.write(text_test)
f.close()

In [50]:
en_sents_test = open('opus.en-ru-test.en').read().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().splitlines()

In [89]:
res = {}
for i, pair in enumerate(list(zip(en_sents_test[:100], ru_sents_test[:100]))):
  t, s = pair
  print(s, t, sep=' --> ')
  score = nltk.translate.bleu_score.sentence_bleu([t], translate(s), auto_reweigh=True)
  print(score)
  print()
  res[i] = score

Только бы не вылететь. --> If you only stay there.
5.609571844608773e-155

И как ты только справляешься, папа, таская эти коробки взад-вперед целый день. --> I don't know how you do it, Pop, carrying these boxes around every day.
0.2823401832456783

Возможно, у нас есть небольшое преимущество в переговорах. --> We might have a slight edge in mediation.
0.22422597920189083

Сколько времени вы будете делать то, что ему нужно? --> How long is it going to take you to get him what he needs?
0.27920785561656575

1 апреля Президент НКР Бако Саакян принял начальника Генштаба Вооруженных сил Республики Армения генерал-полковника Юрия Хачатурова. --> On 1 April President of the Nagorno Karabagh Republic Bako Sahakyan met head of the General Staff of the Republic of Armenia's Armed forces colonel-general Yuri Khachaturov.
0.24936687902934337

Г-н Приснер также упомянул, что система электронного правосудия не только позволила улучшить процесс ведения дел, но также способствует значительному упоряд

In [95]:
top_5 = sorted(res.items(), key=lambda item: item[1], reverse=True)[:5]
top_5

[(54, 0.6030758354585847),
 (12, 0.534281221398097),
 (47, 0.5114791061530924),
 (98, 0.49331757145281036),
 (48, 0.4858037505398287)]

In [96]:
for i, score in top_5:
  print(ru_sents_test[i], en_sents_test[i], sep=' --> ')
  print(score)

Не могу вспомнить, почему я здесь. --> I can't remember why I'm here. (Coughs)
0.6030758354585847
Коллекция администрации Адамса. --> A collection from the Adams administration.
0.534281221398097
Now, if you have kids yourself, you know what-- --> Now, if you have kids yourself, you know what--
0.5114791061530924
Ты выбросил моё будущее. --> You threw away my future.
0.49331757145281036
В первом полугодии их получили 117 425 беременных женщин. --> In the first half of the year, 117,425 pregnant women received them.
0.4858037505398287


Получились на удивление хорошие переводы


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

Backtranslation это способ увеличить объем обучающего параллельного корпуса в услових, когда параллельных текстов мало, например, когда в языковой паре один из языков малоресурсный.

Такой метод состоит из нескольких этапов. Сначала мы обучаем модель перевода в обратном порядке: если изначальная задача состоит в том, чтобы перевести текст с английского на русский, то мы должны обучить модель переводу с русского на английский. Затем с помощью этой модели мы переводим предложения на языке перевода из моноязычного корпуса -- так, к нашему параллельному прибавляются новые переводы. После этого мы можем обучить модель в должном направлении (англмйский -> русский) на уже расширенном датасете, который состоит из параллельного корпуса, который у нас был изначально, и корпуса, который мы синтезировали из моноязычного корпуса.

Чтобы применить метод backtranslation к данным из этого блокнота, нужно:

1) загрузить датасеты на обоих языках и обучить токенизатор(ы) (к тексту на русском нужно также добавить теги начала и окончания предложения). Если мы ограничиваемся только данными из семинара, то из изначального датасета откладываем в отдельную переменную предложения на русском без эквивалентов на английском.

2) разделить датасеты с параллельными предложениями на трейн и тест

3) обучить модель перевода на параллельных предложениях в направлении русский -> английский

4) с помощью этой модели генерируем переводы на английский для предожений на русском, которые мы отложили в отдельную переменную на шаге 1

5) объединяем оба параллельных датасета -- изначальный и синтезированный в один

6) обучаем новую модель на расширенном датасете в направлении английский -> русский